In [1]:
import networkx as nx

In [2]:
# Load the network from the GraphML file
G = nx.read_graphml('bioneer.graphml')

In [5]:
# Function to calculate homophily based on a given attribute
def calculate_homophily(graph, attribute):
    same_attr_edges = 0
    total_edges = 0

    for node in graph:
        node_attr_value = graph.nodes[node].get(attribute, None)
        if node_attr_value is not None:
            for neighbor in graph.neighbors(node):
                neighbor_attr_value = graph.nodes[neighbor].get(attribute, None)
                if neighbor_attr_value is not None:
                    total_edges += 1
                    if node_attr_value == neighbor_attr_value:
                        same_attr_edges += 1

    homophily_ratio = same_attr_edges / total_edges if total_edges > 0 else None
    return homophily_ratio


# Calculate homophily for 'country'
country_homophily = calculate_homophily(G, 'country')

# Calculate homophily for 'Modularity Class'
modularity_class_homophily = calculate_homophily(G, 'Modularity Class')

country_homophily, modularity_class_homophily

(0.30701754385964913, 0.6260964912280702, None)